In [1]:
import pandas as pd
import numpy as np
import os
from sodapy import Socrata
from tqdm import tqdm
from datetime import datetime, date, time

## Get initial ground truth occupancy

In [2]:
directory = './data/ground_truth/sdot/2022_01'

In [3]:
occ = pd.DataFrame()

In [4]:
col_name = None
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if not filename.startswith('.') and os.path.isfile(f):
        df = pd.read_csv(f)
        if len(occ.columns) == 0 or df.columns.values.tolist() == occ.columns.values.tolist():
            occ = pd.concat([occ, df], ignore_index=True)

In [5]:
def convert_time(timestr):
    try:
        return datetime.strptime(timestr, '%I%p').time()
    except:
        return np.nan
    
def convert_date(datestr):
    try:
        return datetime.strptime(datestr, '%m/%d/%Y').date()
    except:
        return np.nan

In [6]:
occ = occ.drop(columns=['Unnamed: 0','Route','Label','Load Zone','RPZ for that Block','ADA','Rideshare','ILLEGAL','Notes','Pictures'])
occ['Time'] = occ.apply(lambda r: convert_time(r['Time']), axis=1)
occ['Date'] = occ.apply(lambda r: convert_date(r['Date']), axis=1)
occ.dropna(subset=['Time','Date','Element Key'], inplace=True)
occ['Element Key'] = occ.apply(lambda r: int(r['Element Key']), axis=1)
occ = occ.sort_values(by=['Element Key', 'Date', 'Time'])

In [7]:
occ.head()

,Time,Date,Element Key,Location,Side of Street,PAID
3274,09:00:00,2021-10-27,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,5
3275,10:00:00,2021-10-27,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,5
3276,11:00:00,2021-10-27,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,5
3277,12:00:00,2021-10-27,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,5
3278,13:00:00,2021-10-27,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,5


In [8]:
grouped = occ.groupby(['Element Key','Date'])
groups = grouped.filter(lambda r: len(r.index) == 12)

In [9]:
groups = groups.reset_index(drop=True)
groups = groups.drop(columns=["Location", "Side of Street"])
groups["Time"] = pd.to_datetime(groups["Time"], format="%X").dt.hour
groups.rename(columns={"Element Key":"sourceelementkey", "PAID":"paidoccupancy"}, inplace = True)
groups = groups.reindex(columns=["sourceelementkey", "Date", "Time", "paidoccupancy"])
groups.columns = ["sourceelementkey", "date", "hour", "paidoccupancy"]
groups

,sourceelementkey,date,hour,paidoccupancy
0,1013,2022-04-29,8,2
1,1013,2022-04-29,9,0
2,1013,2022-04-29,10,1
3,1013,2022-04-29,11,0
4,1013,2022-04-29,12,1
...,...,...,...,...
9631,136322,2022-04-27,15,5
9632,136322,2022-04-27,16,3
9633,136322,2022-04-27,17,1
9634,136322,2022-04-27,18,0


In [10]:
block_data = groups.drop_duplicates(subset=["date", "sourceelementkey"], keep="first")
block_data = block_data.drop(columns=["hour", "paidoccupancy"])
block_data

,sourceelementkey,date
0,1013,2022-04-29
12,1013,2022-08-22
24,1014,2022-04-29
36,1014,2022-08-22
48,1017,2022-05-03
...,...,...
9576,123943,2022-08-22
9588,131238,2022-05-05
9600,136041,2022-04-29
9612,136041,2022-08-22


## Query naive occupancy from SDOT

In [ ]:
client = Socrata("data.seattle.gov", None)

In [ ]:
def run_query(client, data_id, element_key, date):
    query = "select occupancydatetime, paidoccupancy, sourceelementkey, parkingspacecount where sourceelementkey = " + str(element_key) + " and occupancydatetime between '" + str(date) + "T08:00:00' and '" + str(date) + "T19:59:00'"
    results = client.get(data_id, query=query)
    return results

In [ ]:
query_results = [run_query(client, "bwk6-iycu", element_key, date) for element_key, 
                 date in zip(tqdm(block_data['sourceelementkey']), block_data['date'])]

### Filter blocks datetime pair that does not have full 08-18 period

In [ ]:
occ = pd.DataFrame()
for group in query_results:
    if len(group) == 720:
        df = pd.DataFrame.from_records(group)
        occ = pd.concat([occ, df])
        
occ["occupancydatetime"] = pd.to_datetime(occ["occupancydatetime"], format="%Y-%m-%dT%X.000")

In [55]:
#occ.to_csv("data/sdot_12_raw.csv",index=False)
sdot_raw = pd.read_csv('data/sdot_12_raw.csv')

In [56]:
sdot_raw["occupancydatetime"] = pd.to_datetime(sdot_raw["occupancydatetime"], format="%Y-%m-%d %X")
naive_occ = sdot_raw.groupby(["sourceelementkey", sdot_raw["occupancydatetime"].dt.date, 
                              sdot_raw["occupancydatetime"].dt.hour, 
                              "parkingspacecount"])["paidoccupancy"].agg(lambda x: pd.Series.mode(x)[0]).reset_index(allow_duplicates=True)
naive_occ["paidoccupancy"] = naive_occ["paidoccupancy"].astype(int)/naive_occ["parkingspacecount"].astype(int)

In [57]:
naive_occ.columns = [naive_occ.columns[0], "date", "hour", naive_occ.columns[3], naive_occ.columns[4]]
naive_occ

,sourceelementkey,date,hour,parkingspacecount,paidoccupancy
0,1013,2022-04-29,8,4,0.250000
1,1013,2022-04-29,9,4,0.250000
2,1013,2022-04-29,10,4,0.500000
3,1013,2022-04-29,11,4,0.000000
4,1013,2022-04-29,12,4,0.250000
...,...,...,...,...,...
7147,136322,2022-04-27,15,7,0.285714
7148,136322,2022-04-27,16,7,0.142857
7149,136322,2022-04-27,17,7,0.000000
7150,136322,2022-04-27,18,7,0.000000


In [58]:
cap = naive_occ[["sourceelementkey", "date", "parkingspacecount"]]
cap = cap.drop_duplicates()
cap = cap.reset_index(drop=True)
cap["date"] = cap["date"].astype('string')
cap

,sourceelementkey,date,parkingspacecount
0,1013,2022-04-29,4
1,1013,2022-08-22,4
2,1021,2022-05-03,9
3,1022,2022-05-03,9
4,1037,2022-05-05,12
...,...,...,...
591,123942,2022-08-22,5
592,123943,2022-04-29,10
593,123943,2022-08-22,10
594,131238,2022-05-05,2


In [59]:
naive_occ = naive_occ.drop("parkingspacecount", axis="columns")
#naive_occ.to_csv("data/sdot_12_naive.csv",index=False)

In [60]:
naive_tup = [(key, date) for key, date in zip(naive_block["sourceelementkey"], naive_block["date"])]

In [61]:
gt_occ = groups
gt_occ["date"] = gt_occ["date"].astype('string')

In [62]:
gt_occ = gt_occ[pd.Series(list(zip(gt_occ["sourceelementkey"], gt_occ["date"]))).isin(naive_tup)].reset_index(drop=True)
gt_occ

,sourceelementkey,date,hour,paidoccupancy
0,1013,2022-04-29,8,2
1,1013,2022-04-29,9,0
2,1013,2022-04-29,10,1
3,1013,2022-04-29,11,0
4,1013,2022-04-29,12,1
...,...,...,...,...
7147,136322,2022-04-27,15,5
7148,136322,2022-04-27,16,3
7149,136322,2022-04-27,17,1
7150,136322,2022-04-27,18,0


In [63]:
gt_occ = pd.merge(gt_occ, cap, on=['sourceelementkey', 'date'])
gt_occ["paidoccupancy"] = gt_occ["paidoccupancy"].astype(int)/gt_occ["parkingspacecount"].astype(int)
gt_occ = gt_occ.drop("parkingspacecount", axis="columns")
gt_occ

,sourceelementkey,date,hour,paidoccupancy
0,1013,2022-04-29,8,0.500000
1,1013,2022-04-29,9,0.000000
2,1013,2022-04-29,10,0.250000
3,1013,2022-04-29,11,0.000000
4,1013,2022-04-29,12,0.250000
...,...,...,...,...
7147,136322,2022-04-27,15,0.714286
7148,136322,2022-04-27,16,0.428571
7149,136322,2022-04-27,17,0.142857
7150,136322,2022-04-27,18,0.000000


In [64]:
gt_occ.to_csv("data/sdot_12_gt.csv", index=False)